In [1]:
# Setup: imports and paths
import os
import numpy as np
import pandas as pd

from gaitsetpy.dataset.daphnet import DaphnetLoader
from gaitsetpy.features.gait_features import extract_gait_features
from gaitsetpy.classification.models import get_classification_model

DATA_DIR = os.path.join(os.path.dirname(".."), "data")
print(f"Data directory: {os.path.abspath(DATA_DIR)}")


Data directory: c:\Users\ahars\Desktop\Funstuff\coding\alohomora_labs\gaitSetPy\examples\notebooks\data


In [2]:
# 1) Load Daphnet dataset
loader = DaphnetLoader()
data, names = loader.load_data(DATA_DIR)
print(f"Loaded {len(data)} subjects. Example file: {names[0] if names else 'N/A'}")

# Quick peek
if data:
    display(data[0].head())


Download Progress: 21.4MiB [00:08, 2.60MiB/s]


Download completed successfully! File saved to: data\daphnet.zip
Loaded 17 subjects. Example file: S01R01.txt


,shank,shank_h_fd,shank_v,shank_h_l,thigh,thigh_h_fd,thigh_v,thigh_h_l,trunk,trunk_h_fd,trunk_v,trunk_h_l,annotations
time,,,,,,,,,,,,,
15,973.304166,70,39,-970,0.0,0,0,0,0.0,0,0,0,0
31,973.304166,70,39,-970,0.0,0,0,0,0.0,0,0,0,0
46,963.120449,60,49,-960,0.0,0,0,0,0.0,0,0,0,0
62,963.120449,60,49,-960,0.0,0,0,0,0.0,0,0,0,0
78,962.091991,50,39,-960,0.0,0,0,0,0.0,0,0,0,0


In [3]:
# 2) Create sliding windows
windows = loader.create_sliding_windows(data, names, window_size=192, step_size=32)
print(f"Created sliding windows for {len(windows)} subjects.")

# Flatten windows into a list of sensor windows for feature extraction
sensor_windows = []
for subject in windows:
    for sensor in subject['windows']:
        sensor_windows.append(sensor)

print(f"Prepared {len(sensor_windows)} sensor windows.")


Created sliding windows for 17 subjects.
Prepared 221 sensor windows.


In [ ]:
# 3) Extract gait features
fs = loader.metadata.get('sampling_frequency', 64)
features = extract_gait_features(sensor_windows, fs, time_domain=True, frequency_domain=True, statistical=True, verbose=True)
print(f"Extracted features for {len(features)} sensors.")

# Prepare feature_dicts grouped by sensor, keep lists per-window
feature_dicts = []
annotation_sensor = None
for sensor in features:
    if sensor['name'] == 'annotations':
        annotation_sensor = sensor
        break

for sensor in features:
    if sensor['name'] != 'annotations':
        n_windows = len(next(iter(sensor['features'].values())))
        feature_dicts.append({
            'name': sensor['name'],
            'features': sensor['features'],
            'annotations': annotation_sensor['annotations'][:n_windows] if annotation_sensor else [0]*n_windows
        })

print(f"Prepared {len(feature_dicts)} feature dictionaries for classification.")


🚀 GaitFeatureExtractor initialized successfully!
📊 Default configuration: {'time_domain': True, 'frequency_domain': True, 'statistical': True, 'ar_order': 3}

🔍 STARTING GAIT FEATURE EXTRACTION
📈 Total sensors/windows to process: 221
🔊 Sampling frequency: 64 Hz
⏱️  Time domain features: ✅
🌊 Frequency domain features: ✅
📊 Statistical features: ✅
🔄 Auto-regression order: 3
------------------------------------------------------------


🔍 Processing Sensors:   0%|          | 0/221 [00:00<?, ?sensor/s, Current=shank, Windows=2895]INFO:gaitsetpy.features.gait_features:🎯 Processing sensor: shank
INFO:gaitsetpy.features.gait_features:📦 Number of windows: 2895


  ⏱️  Extracting time domain features for shank...
    🔍 Computing time domain features...


  ✅ Time domain: 12 feature types, 34740 total features
  🌊 Extracting frequency domain features for shank...
    🔍 Computing frequency domain features...


  ✅ Frequency domain: 8 feature types, 23160 total features
  📊 Extracting statistical features for shank...
    🔍 Computing statistical features...


...

🔍 Processing Sensors:   2%|▏         | 4/221 [00:58<52:21, 14.48s/sensor, Current=thigh, Windows=2895]    INFO:gaitsetpy.features.gait_features:🎯 Processing sensor: thigh
INFO:gaitsetpy.features.gait_features:📦 Number of windows: 2895


  ✅ Auto-regression: 1 feature types, 2895 total features
  🎯 Total features extracted for shank_h_l: 75270
  📋 Feature types: ['mean', 'std', 'variance', 'rms', 'range', 'median', 'mode', 'mean_absolute_value', 'median_absolute_deviation', 'peak_height', 'zero_crossing_rate', 'energy', 'dominant_frequency', 'peak_frequency', 'power_spectral_entropy', 'principal_harmonic_frequency', 'stride_times', 'step_time', 'cadence', 'freezing_index', 'skewness', 'kurtosis', 'entropy', 'interquartile_range', 'correlation', 'ar_coefficients']
----------------------------------------
  ⏱️  Extracting time domain features for thigh...
    🔍 Computing time domain features...


  ✅ Time domain: 12 feature types, 34740 total features
  🌊 Extracting frequency domain features for thigh...
    🔍 Computing frequency domain features...


  ✅ Frequency domain: 8 feature types, 23160 total features
  📊 Extracting statistical features for thigh...
    🔍 Computing statistical features...


      🔗 Computing correlation features...
  ✅ Statistical: 5 feature types, 14475 total features
  🔄 Extracting auto-regression coefficients for thigh...
    🔍 Computing auto-regression coefficients (order=3)...


In [ ]:
# 4) Train and evaluate multiple models
model_names = ['random_forest', 'mlp', 'lstm', 'bilstm', 'cnn']

for model_name in model_names:
    print(f"\n=== Training model: {model_name.upper()} ===")
    try:
        model = get_classification_model(model_name)
    except Exception as e:
        print(f"Skipping {model_name}: {e}")
        continue
    try:
        model.train(feature_dicts, test_size=0.2, validation_split=True)
        results = model.evaluate(feature_dicts, detailed_report=True)
        print(f"Results for {model_name}:\n", {k: (v if k!='classification_report' else 'see detailed report') for k, v in results.items()})
    except Exception as e:
        print(f"Error training/evaluating {model_name}: {e}")
